# Fundamentals of Information Systems

## Python Programming (for Data Science)

### Master's Degree in Data Science

#### Giorgio Maria Di Nunzio
#### (Courtesy of Gabriele Tolomei FIS 2018-2019)
<a href="mailto:giorgiomaria.dinunzio@unipd.it">giorgiomaria.dinunzio@unipd.it</a><br/>
University of Padua, Italy<br/>
2021/2022<br/>

# Lecture 13: The Classification Problem - Example (Part 2)

## Instructions

-  We consider the dataset file <code>**dataset.csv**</code>, which is contained in the <code>**loan-prediction**</code> directory on the Moodle page.

-  A description of the dataset is available in the <code>**README.txt**</code> file on the same directory.

-  **GOAL:** Use information from past loan applicants contained in <code>**dataset.csv**</code> to predict whether a _new_ applicant should be granted a loan or not.

In [ ]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Import stats module from scipy, which contains a large number 
# of probability distributions as well as an exhaustive library of statistical functions.
import scipy.stats as stats

%matplotlib inline

## Summary of Part 1

In [ ]:
# Path to the local dataset file
DATASET_PATH = "./data/loan-prediction/dataset.csv"

## Loading the Dataset

In [ ]:
# Load the dataset with Pandas
data = pd.read_csv(DATASET_PATH, sep=",", index_col="Loan_ID")
print("Shape of the dataset: {}".format(data.shape))
data.head()
# NOTE: the first line of the file is considered as the header

## Handling Missing Values (NA)

Need to have the least amount of missing values. We can discard the record or the fieature. This can be harmful cuz we are removing information and dataset which could and will help us to reduce to in and out sample error.

Simplest thing to do is to set the missing values to the avg value to tha specific feature. I.e. the age of a person we have a value that will be equally distant to each other value.

Shortest way to creat function, lambda function. Given argument x, if the column is numeric, fill the empty value with the median value of the column. Media is better than mean cuz it is very sensitive to extreme value while mean is exactly the same. If it is not a numeric type, you can use the mode of the columnm and iloc[0]. The elemnt of a catgeircal value of the frequen element


In [ ]:
# is_numeric_dtype(pandas.Series) returns True iff the dtype associated with the pandas.Series is numeric
from pandas.api.types import is_numeric_dtype

data = data.apply(lambda x: x.fillna(x.median()) 
                      if is_numeric_dtype(x) 
                      else x.fillna(x.mode().iloc[0]))

## Handling Outliers

Extreme values ar not necessarily a negative thing. If thei are real. In the loan prediction is like a billionaire. Sometimes we get a billione inside all the avg salaries. Will change the avgs but it is part of dataset.

Another thing is whan you have extreme value but it seems to be a mistake. Alway a good wait to choose the models that are the simplest ones, thei not vary to much given extreme cases

We can keep them as they are and rely on the dataset or we try to reduce the extremes, like if you think in terms of ...

Don leve the transformation without comments kuz you are effectibely schrinking data, compressing them


In [ ]:

# Let's winsorize 'ApplicantIncome', 'CoapplicantIncome', and 'LoanAmount'
stats.mstats.winsorize(data.ApplicantIncome, limits=0.05, inplace=True)
stats.mstats.winsorize(data.CoapplicantIncome, limits=0.05, inplace=True)
stats.mstats.winsorize(data.LoanAmount, limits=0.05, inplace=True)

# Apply log-transformation to 'ApplicantIncome' and assign it to a new column
data['Log_ApplicantIncome'] = data.ApplicantIncome.apply(np.log)
# Apply log-transformation to 'LoanAmount' and assign it to a new column
data['Log_LoanAmount'] = data.LoanAmount.apply(np.log)

## Encoding Categorical Features: One-Hot Encoding

All the models need numerical amtrix, we will do these tranformation. How to transform something not numerical to something that is. It has a name, one-hot encoding. Suppose we have a column with label 1. I assign every lable with the corrispongin number. 

What is the meaning of the 2d space of the 20/30 etc. What if instead of 20/30... er have 200/300. We are placing pints in a two dim space. This will a ffect the traingin, the KNN will be also affected. Corrispondance between label and a number is very diffucit cuz is diffciult to make sanes to what to do with the number. If we change the order the labels a dn mapping to numbers is complietly differe.

We ar not going to do this. We are gouing to transform column2 (the one with the label), to more columns. One for every unique value. So with 4 labels, we will have 4 new columns (with 100, there will be 100 cols). To fill the column, we will set 1 for every correspondiv value, 0 otherwise. If the 1st value has label1 (and there are label 1 through 4) the column label 1 will have one, teh other three will have 0 value. 

In pandase, using the function get dummies. Stupid variable generated to structurize the space. 


In [ ]:
# In pandas we can achieve easily one-hot encoding using the 'get_dummies()' function
categorical_features = [col for col in data.columns if not is_numeric_dtype(data[col]) and col != 'Loan_Status']
data_with_dummies = pd.get_dummies(data, columns = categorical_features)
data_with_dummies.head()

Pop function remove from one column and returns it. Insert the elements in the last column 

In [ ]:
# Just as a convention, I prefer to place the column to be predicted
# as the last one.
columns = data_with_dummies.columns.tolist()

# Popping out 'Loan_Status' from the list and insert it back at the end.
columns.insert(len(columns), columns.pop(columns.index('Loan_Status')))

# Let's refactor the DataFrame using this new column index
data_with_dummies = data_with_dummies.loc[:, columns]
data_with_dummies.head()

## Encoding Binary Class Label

In this case, we are re-labeling the output 


In [ ]:
data = data_with_dummies
data.Loan_Status = data.Loan_Status.map(lambda x: 1 if x=='Y' else -1)
data.head()

# 4. Building a Predictive Model

In [ ]:
from sklearn.metrics import SCORERS
from sklearn.feature_extraction import DictVectorizer as DV
from sklearn import tree
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import explained_variance_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestRegressor
#from sklearn.externals import joblib

## 4.1 Splitting the Dataset: _Training_ vs. _Test_

Thi line says: take all the columns except the last one

In [ ]:
"""
Extract the feature matrix from our original DataFrame.
"""
# Feature matrix X is composed of all the columns 
# except 'Loan_Status' (i.e., the target class label)
X = data.iloc[:, :-1]
X.head()

Separate X and y cuz this i how sklearn works

In [ ]:
"""
Similarly, we want to extract the target class column vector y.
"""
y = data.Loan_Status
y.head()

In [ ]:
"""
Let's split our dataset with scikit-learn 'train_test_split' function, 
which splits the input dataset into training and test set, respectively.
We want the training set to account for 80% of the original dataset, whilst 
the test set to account for the remaining 20%.
Additionally, we would like to take advantage of stratified sampling,
so as to obtain the same target distribution in both the training and the test sets.
"""

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=43, 
                                                    stratify=y)

In [ ]:
print("Training Set shape: {}".format(X_train.shape))
print("Test Set shape: {}".format(X_test.shape))

## Feature Scaling: Why/When

-  <span style="color: red">**REMEMBER:**</span> Not every learning models are sensitive to different feature scales! 

-  For example, in the case of Logistic Regression the vector of model parameters we come up with when we minimize the negative log-likelihood - using gradient descent (iterative) solution - is **not** affected by different feature scales, except for a constant.

-  You can convince yourself of this by computing the gradient of the negative log-likelihood using non-scaled and scaled features.

-  Other models, instead, are not invariant with respect to scalar transformations of the input (features), and leads to completely different results if features are not properly scaled (e.g., Support Vector Machines or SVM). 

## Feature Scaling: How

-  Feature scaling **cannot** be done looking at the whole dataset!

-  In other words, either you standardize (using **z-scores**) or normalize (using **min-max**) your features you **must** do it considering only the training set portion of your dataset.

-  The same scaling, then, should be applied to the test set.

In [ ]:
"""
Let's use two different feature scaling strategies: standard z-scores and min-max
"""
# The following is the scikit-learn package which provides
# various preprocessing capabilities
from sklearn import preprocessing

We can scale according to a standard scaler. Is a conventient functon. FInd the min of the values of the feature, find std dev, use the formula line 7 to 

Ore we can use the minimax scaler, find max and min value fo the feature and than make a scale devided by the difference between max and min. Is it mandatory to scale things, no, but it may be effective. In terms of performance, the only answer is to try it and then we will see compared to model without scaling

In [ ]:
# Standardizing features using z-score
std_scaler = preprocessing.StandardScaler().fit(X_train)
X_train_std = std_scaler.transform(X_train)
# Alternatively, using pure pandas:
# X_train_mean = X_train.mean()
# X_train_std = X_train.std()
# X_train_std = (X_train - X_train_mean)/X_train_std

# Normalizing features using min-max
minmax_scaler = preprocessing.MinMaxScaler().fit(X_train)
X_train_minmax = minmax_scaler.transform(X_train)
# Alternatively, using pure pandas:
# X_train_max = X_train.max()
# X_train_min = X_train.min()
# X_train_minmax = (X_train - X_train_min)/(X_train_max - X_train_min)

At this stage, we can work with 3 different feature matrices:
- The original one: X_train
- The standardized one: X_train_std
- The min-max normalized one: X_train_minmax

In the following, however, we work only on the original feature matrix X_train



How are we avluating the model? Lots of perfomarmance measure in sklearn. Not gonna ask to implemnt recalla nd precision. THe thing is to make sure how these measure work 

In [ ]:
"""
General function used to assess the quality of predictions
in terms of two scores: accuracy and ROC AUC (Area Under the ROC Curve)
"""
def evaluate(true_values, predicted_values):
    
    # Classification Accuracy
    print("Accuracy = {:.3f}".
          format(accuracy_score(true_values, predicted_values)))
    
    # Explained variance score: 1 is perfect prediction
    print("Area Under the ROC Curve (ROC AUC) = {:.3f}".
          format(roc_auc_score(true_values, predicted_values)))

In [ ]:
# Create logistic regression object
model = LogisticRegression(solver = "liblinear")

# 1. Try to fit this logistic regressor to our original training set
model.fit(X_train, y_train)

# 2. Assess the quality of predictions made on the same training set
print("***** Evaluate Performance on Training Set *****")
evaluate(y_train, model.predict(X_train))
print()

# 3. Assess the quality of predictions made on the test set
print("***** Evaluate Performance on Test Set *****") 
evaluate(y_test, model.predict(X_test))

For class -1, people who get the loan, 

<!-- 
Il prof ha sbagliato adire te robe.

Whe the model says the person will get the loan, it will be very precise, vry like a person that a preso get a loan for sure. ONly 7/100 the model makes mistake. Problem is that model says no to lot of people who wold need th loan. Very low recall for people who needs a loan.

Easyear to classify to say no than yes cuz you have more datapoints about the negative class -->


The optimization wnet to the other way aroind to usually would see. Optimize high recall form positive class. Basically saying yes to evrityingh, is like a perfect acceptor. Including a loto of potential people who do not have to get the loan. What is the situation that is more risky: better o give loan to people who shouldng or is it to risky the otherway: this is the decision theory where to put the threshold .



In [ ]:
print(classification_report(y_test, model.predict(X_test)))

Having just one test set an trainign set is not good. A better way is better to use the simplest cross validation. 

This returns result for each situation fold. IN three folds out of the the accuracy

Fermormance that goes from 67 upt to 86 precent. We have 20 percent to accuracy different which s a lot. We have no idea which is better: it is a distribution of accuracies. 


In [ ]:
# Simplest usage of cross-validation
model = LogisticRegression(solver = "liblinear")
cv = cross_validate(model, X, y, cv=10, scoring=('roc_auc', 'accuracy'), return_train_score=True)
pd.DataFrame(cv)

In [ ]:
# Model evaluation using cross-validation
print("***** Evaluate Average Performance on Training Set *****")
print("Avg. Training Set Accuracy = {:.3f}".format(np.mean(cv['train_accuracy'])))
print("Avg. Training Set ROC AUC = {:.3f}".format(np.mean(cv['train_roc_auc'])))
print()
print("***** Evaluate Average Performance on Cross-Validation Set *****")
print("Avg. Test Set Accuracy = {:.3f}".format(np.mean(cv['test_accuracy'])))
print("Avg. Test Set ROC AUC = {:.3f}".format(np.mean(cv['test_roc_auc'])))

In [ ]:
# Define an object of type KFold and pass it to the cross_validate function
model = LogisticRegression(solver = "liblinear")

k_fold = KFold(n_splits=10, shuffle=True, random_state=42)

cv = cross_validate(model, X, y, cv=k_fold, scoring=('roc_auc', 'accuracy'), return_train_score=True)
print(cv)

In [ ]:
k_fold

In [ ]:
# Model evaluation using cross-validation
print("***** Evaluate Average Performance on Training Set *****")
print("Avg. Training Set Accuracy = {:.3f}".format(np.mean(cv['train_accuracy'])))
print("Avg. Training Set ROC AUC = {:.3f}".format(np.mean(cv['train_roc_auc'])))
print()
print("***** Evaluate Average Performance on Cross-Validation Set *****")
print("Avg. Test Set Accuracy = {:.3f}".format(np.mean(cv['test_accuracy'])))
print("Avg. Test Set ROC AUC = {:.3f}".format(np.mean(cv['test_roc_auc'])))

In [ ]:
# Define an object of type StratifiedKFold and pass it to the cross_validate function
model = LogisticRegression(solver = "liblinear")

k_fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=37)

cv = cross_validate(model, X, y, cv=k_fold, scoring=('roc_auc', 'accuracy'), return_train_score=True)
print(cv)

In [ ]:
# Model evaluation using cross-validation
print("***** Evaluate Average Performance on Training Set *****")
print("Avg. Training Set Accuracy = {:.3f}".format(np.mean(cv['train_accuracy'])))
print("Avg. Training Set ROC AUC = {:.3f}".format(np.mean(cv['train_roc_auc'])))
print()
print("***** Evaluate Average Performance on Cross-Validation Set *****")
print("Avg. Test Set Accuracy = {:.3f}".format(np.mean(cv['test_accuracy'])))
print("Avg. Test Set ROC AUC = {:.3f}".format(np.mean(cv['test_roc_auc'])))

## Model Selection and Evaluation

-  So far, we have just focused on a very specific _instance_ of a Logistic Regression model.

-  In other words, we haven't spent time trying to _tune_ any "meta-parameter" (known as **hyperparameter**) of our model.

-  In practice, we used default values of hyperparameters for our Logistic Regression model, according to <code>**scikit-learn**</code>

-  We didn't perform any actual model selection, as hyperparameters are fixed!

-  The figures we output for test accuracy/ROC AUC scores are our estimates of _generalization_ performance of our model (i.e., evaluation)

## Model Selection and Evaluation (cont'd)

-  Most of the time, though, we may need to do one of the following:
    1.  Fix a "family" of models (e.g., Logistic Regression) and perform hyperparameter selection;
    2.  Choose between a set of models (e.g., Logistic Regression, SVM, Decision Tree, etc.), each one with a fixed (i.e., default) set of hyperparameters;
    3.  A mixture of the above, where we have to select the best hyperparameters of the best model picked from a set of different models.

-  In any case, we also need to provide an estimate of the generalization performance of the chosen model.

## Case 1: Select Best Hyperparameters of a Fixed Family of Models

## 1.1: Using Validation Set

Dictionary, for each model we have a listo of all the hyperparameters

Logistic rgression with one fixed hypermarameter, which is the algorhtm to solve Then a dictionary to hyperparameter values for the parameter C.


In [ ]:
models_and_hyperparams = {'LogisticRegression': (LogisticRegression(solver = "liblinear"),
                                                 {'C': [0.01, 0.05, 0.1, 0.5, 1, 2]}
                                                )
                         }

Outer splitting, separates trining and testing. Test meaning th portion of the dataset that will use to produce the iutput of the perofmrnce of the model once you optimize it. 

Inner splitting, is just a re applicaiton of he splitting. A way to produce a kind of  

In [ ]:
# Outer splitting: Training vs. Test set (e.g., 80÷20) used to separate training-selection-evaluation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=73, 
                                                    stratify=y)

# Inner splitting (i.e., within the outer training set): Training vs. Validation (e.g., 80÷20)
# Training set is used to train the model, validation set is used to select the best hyperparameters
X_train_train, X_validation, y_train_train, y_validation = train_test_split(X_train, y_train, test_size=0.2, 
                                                    random_state=42, 
                                                    stratify=y_train)

In [ ]:
# Keep the training score obtained with each hyperparameter
training_scores = {}
# Keep the validation score obtained with each hyperparameter
validation_scores = {}
# Keep only the best training/validation scores
best_training_score = {}
best_validation_score = {}

# Get the only model available
model = models_and_hyperparams['LogisticRegression'][0]
# Get the hyperparameters for that model
hyperparams = models_and_hyperparams['LogisticRegression'][1]

# Loop through all the hyperparameters
for hp in hyperparams:
    training_scores[hp] = {}
    validation_scores[hp] = {}
    
    # Loop through all the value of a specific hyperparameter
    for val in hyperparams[hp]:
        # set the model's hyperparameter to the current value
        model.set_params(**{hp: val})
        
        # fit the model on the inner training portion 
        model.fit(X_train_train, y_train_train)
        
        # store the inner training score
        training_score = accuracy_score(y_train_train, model.predict(X_train_train))
        training_scores[hp][val] = training_score
        
        # store the inner validation score
        validation_score = accuracy_score(y_validation, model.predict(X_validation))
        validation_scores[hp][val] = validation_score
        
        # Update best training/validation scores
        if not best_training_score:
            best_training_score[hp] = (val, training_score)
        else:
            if best_training_score[hp][1] < training_score:
                best_training_score[hp] = (val, training_score)
                
        if not best_validation_score:
            best_validation_score[hp] = (val, validation_score)
        else:
            if best_validation_score[hp][1] < validation_score:
                best_validation_score[hp] = (val, validation_score)

Across of all possible values, the best one in the validation set is 0.5. What do we do now? Now that we have found the best value in the hyperparamenter C, we want to use the entire trining set for the model

In [ ]:
print("***** Evaluate Performance on Training Set *****")
print(training_scores)
print("***** Evaluate Performance on Validation Set *****")
print(validation_scores)
print("***** Best Accuracy Score on Training Set *****")
print(best_training_score)
print("***** Best Accuracy Score on Validation Set *****")
print(best_validation_score)

pick best parameter in the dictionary we have and reset the model with this hyperparameter. What do we do? Noew can train the model on the entire training ste xtrain. We can use the entire information in the trining set. Once we trained this we can truly test it. that is the piplein for an hold out istuatin. Use hluld out approach when you have reasonable amount of points. No effect random of having one portion of the dataset or the other.  In general we will use crsso validation approach. Pipleine of cross validation but need to introducte the c-valid part. 

In [ ]:
# We set the model's hyperparameters to those leading to the best score on the validation test
best_params = dict([(list(best_validation_score.keys())[0], list(best_validation_score.values())[0][0])])
model.set_params(**best_params)

# We fit this model to the whole training set portion
model.fit(X_train, y_train)
print("***** Evaluate Performance on Training Set *****")
evaluate(y_train, model.predict(X_train))
print("***** Evaluate Performance on Test Set *****")
evaluate(y_test, model.predict(X_test))

## 1.2.a: Using Cross-Validation (Single Hyperparameter)

Define same family with the same hyper parameter of before

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=73, 
                                                    stratify=y)

First we declare the kfold object and even for the kfold cross validaton we can do a stratifiec crss falidation. In each fold thre is the same proportion of classes. 

In [ ]:
models_and_hyperparams = {'LogisticRegression': (LogisticRegression(solver = "liblinear"),
                                                 {'C': [0.01, 0.05, 0.1, 0.5, 1, 2]}
                                                )
                         }

In [ ]:
k_fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Get the only model available
model = models_and_hyperparams['LogisticRegression'][0]

# Get the hyperparameters for that model
hyperparams = models_and_hyperparams['LogisticRegression'][1]

gs = GridSearchCV(estimator=model, param_grid=hyperparams, cv=k_fold, 
                  scoring='accuracy',
                  verbose=True,
                 return_train_score=True)
gs.fit(X_train, y_train)
pd.DataFrame(gs.cv_results_)

In [ ]:
print("Best hyperparameter: {}".format(gs.best_params_))
print("Best accuracy score: {:.3f}".format(gs.best_score_))
evaluate(y_test, gs.predict(X_test))

## 1.2.b: Using Cross-Validation (Multiple Hyperparameters)

In [ ]:
models_and_hyperparams = {'LogisticRegression': (LogisticRegression(solver = "liblinear"),
                                                 {'C': [0.01, 0.05, 0.1, 0.5, 1, 2],
                                                 'penalty': ['l1', 'l2']}
                                                )
                         }

In [ ]:
k_fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=31)

# Get the only model available
model = models_and_hyperparams['LogisticRegression'][0]

# Get the hyperparameters for that model
hyperparams = models_and_hyperparams['LogisticRegression'][1]

gs = GridSearchCV(estimator=model, param_grid=hyperparams, cv=k_fold, 
                  scoring='accuracy',
                  verbose=True,
                 return_train_score=True)
gs.fit(X_train, y_train)
pd.DataFrame(gs.cv_results_)

In [ ]:
print("Best hyperparameter: {}".format(gs.best_params_))
print("Best accuracy score: {:.3f}".format(gs.best_score_))
evaluate(y_test, gs.predict(X_test))

## Case 2: Select Best Model out of a Set of Family of Models with Fixed Hyperparameters

## Using Cross Validation

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=73, 
                                                    stratify=y)

In [ ]:
models = {'LogisticRegression': LogisticRegression(solver = "liblinear", max_iter=1000),
          'LinearSVC': LinearSVC(),
          'DecisionTreeClassifier': DecisionTreeClassifier(),
          'RandomForestClassifier': RandomForestClassifier(),
          'GradientBoostingClassifier': GradientBoostingClassifier()
          # Add other families of models here...
         }

In [ ]:
k_fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
cv_scores = {}
for model_name, model in models.items():
    cv_scores[model_name] = cross_val_score(model, X_train, y_train, cv=k_fold, scoring='accuracy')

In [ ]:
cv_df = pd.DataFrame(cv_scores).transpose()
cv_df

In [ ]:
cv_df['avg_cv'] = np.mean(cv_df, axis=1)
cv_df['std_cv'] = np.std(cv_df, axis=1)
cv_df = cv_df.sort_values(['avg_cv', 'std_cv'], ascending=[False,True])
cv_df

In [ ]:
# Model Selection: Logistic Regression is the best overall method, therefore we pick that!
# Now we need to provide an estimate of its generalization performance. 
# To do so, we evaluate it against the test set portion we previously held out.
model = models[cv_df.index[0]]
# Re-fit the best selected model on the whole training set
model.fit(X_train, y_train)
# Evaluation
print("***** Evaluate Performance on Training Set *****")
evaluate(y_train, model.predict(X_train))
print("***** Evaluate Performance on Test Set *****")
evaluate(y_test, model.predict(X_test))

## Case 3: Select the Best Hyperparameters AND the Best Model from a Family of Models

In [ ]:
models_and_hyperparams = {'LogisticRegression': (LogisticRegression(),
                                                 {'C': [0.01, 0.05, 0.1, 0.5, 1, 2],
                                                 'penalty': ['l1', 'l2']}
                                                ),
                          'RandomForestClassifier': (RandomForestClassifier(),
                                       {'n_estimators': [10, 50, 100]}
                                       ),
                          'DecisionTreeClassifier': (DecisionTreeClassifier(),
                                                     {'criterion': ['gini', 'entropy'], 
                                                      'max_depth': [i for i in range(1, X.shape[1]+1)]}
                                                    )
                         }

In [ ]:
# `outer_cv` creates 10 folds for estimating generalization error
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# when we train on a certain fold, we use a second cross-validation
# split in order to choose hyperparameters
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=73)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=37, 
                                                    stratify=y)

# we will collect the average of the scores on the 10 outer folds in this dictionary
# with keys given by the names of the models in `models_and_hyperparams`
average_scores_across_outer_folds_for_each_model = dict()

# find the model with the best generalization error
for name, (model, params) in models_and_hyperparams.items():
    # this object is a classifier that also happens to choose
    # its hyperparameters automatically using `inner_cv`
    model_optimizing_hyperparams = GridSearchCV(estimator=model, 
                                                param_grid=params,
                                                cv=inner_cv, 
                                                scoring='accuracy',
                                               verbose=True)

    # estimate generalization error on the 10-fold splits of the data
    scores_across_outer_folds = cross_val_score(model_optimizing_hyperparams,
                                                X_train, y_train, cv=outer_cv, scoring='accuracy')

    # get the mean accuracy across each of outer_cv's 10 folds
    average_scores_across_outer_folds_for_each_model[name] = np.mean(scores_across_outer_folds)
    performance_summary = 'Model: {name}\nAccuracy in the 10 outer folds: {scores}.\nAverage Accuracy: {avg}'
    print(performance_summary.format(
        name=name, scores=scores_across_outer_folds,
        avg=np.mean(scores_across_outer_folds)))
    print()

print('Average score across the outer folds: ',
      average_scores_across_outer_folds_for_each_model)

many_stars = '\n' + '*' * 100 + '\n'
print(many_stars + 'Now we choose the best model and refit on the whole dataset' + many_stars)

best_model_name, best_model_avg_score = max(
    average_scores_across_outer_folds_for_each_model.items(),
    key=(lambda name_averagescore: name_averagescore[1]))

# get the best model and its associated parameter grid
best_model, best_model_params = models_and_hyperparams[best_model_name]

# now we refit this best model on the whole dataset so that we can start
# making predictions on other data, and now we have a reliable estimate of
# this model's generalization error and we are confident this is the best model
# among the ones we have tried
final_model = GridSearchCV(best_model, best_model_params, cv=inner_cv)
final_model.fit(X_train, y_train)

print('Best model: \n\t{}'.format(best_model), end='\n\n')
print('Estimation of its generalization performance (accuracy):\n\t{}'.format(
    best_model_avg_score), end='\n\n')
print('Best parameter choice for this model: \n\t{params}'
      '\n(according to cross-validation `{cv}` on the whole dataset).'.format(
      params=final_model.best_params_, cv=inner_cv))


y_true, y_pred, y_pred_prob = y, final_model.predict(X), final_model.predict_proba(X)
print()
print(classification_report(y_true, y_pred))
roc = roc_auc_score(y_true, y_pred_prob[:,1])
acc = accuracy_score(y_true, y_pred)
print("Accuracy = [{:.3f}]".format(acc))
print("Area Under the ROC = [{:.3f}]".format(roc))